# Learning how to Solve a Linear Problem using Neural Networks in PyTorch

Import basic data types

In [1]:
import pandas as pd
import numpy  as np

By building our own GMRES function, we can take it appart and play with it's insides

In [2]:
from src_dir.gmres         import GMRES, matmul_a, mat_to_a
from src_dir.nn_collection import TwoLayerNet
from src_dir.nn_predictor  import nn_preconditioner

Pyplot should display images inline

In [3]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


Work with paths...

In [4]:
from os.path import join

Import image IO libraries -- might not be needed, but I'll keep it in here or now

In [5]:
from skimage.io          import imread
import matplotlib.pyplot as     pp

Creates the validation set

In [6]:
from sklearn.model_selection import train_test_split

Evaluates the model

In [7]:
from sklearn.metrics import accuracy_score
from tqdm            import tqdm, trange

Pytorch Libraries

In [8]:
import torch
from torch.autograd import Variable
from torch.nn       import Linear, ReLU, CrossEntropyLoss, \
                           Sequential, Conv2d, MaxPool2d,  \
                           Module, Softmax, BatchNorm2d, Dropout
from torch.optim    import Adam, SGD

In [9]:
A = np.matrix(
    [[1, 1], 
     [3, -4]]
)

In [10]:
xt = np.array([2,1])
b  = matmul_a(A, xt)
x0 = np.array([0, 0])

In [11]:
b

array([3, 2])

In [12]:
e = 0
nmax_iter = 5

In [13]:
x = GMRES(A, b, x0, e, nmax_iter)

In [14]:
x

[array([3, 2]), array([1.96153846, 1.30769231]), array([2., 1.])]

In [15]:
x=torch.empty(0, 2)
y=torch.randn(1, 2)

In [16]:
x

tensor([], size=(0, 2))

In [17]:
y[0, :]

tensor([-0.2623, -0.4574])

In [18]:
z = torch.cat((x, y), 0)
z

tensor([[-0.2623, -0.4574]])

In [19]:
torch.cat((z, y), 0)

tensor([[-0.2623, -0.4574],
        [-0.2623, -0.4574]])

In [20]:
x = GMRES(A, b, x0, e, nmax_iter)

In [21]:
x

[array([3, 2]), array([1.96153846, 1.30769231]), array([2., 1.])]

In [22]:
@nn_preconditioner(retrain_freq=2, debug=True,InputDim=2,HiddenDim=100,OutputDim=2)
def MLGMRES(A, b, x0, e, nmax_iter):
    return GMRES(A, b, x0, e, nmax_iter)

In [23]:
b1 = np.array([3., 2.])
MLGMRES(A, b1, x0, e, nmax_iter)

[array([3., 2.]), array([1.96153846, 1.30769231]), array([2., 1.])]

In [24]:
b2 = np.array([3.1, 2.1])
MLGMRES(A, b2, x0, e, nmax_iter)

retraining


[array([3.1, 2.1]),
 array([2.00470377, 1.35802513]),
 array([2.07142857, 1.02857143])]

In [25]:
b3 = np.array([3.2, 2.0])
MLGMRES(A, b3, x0, e, nmax_iter)

[array([ 0.02922029, -0.18832302]),
 array([2.11798878, 1.08819891]),
 array([2.11428571, 1.08571429])]

In [26]:
b4 = np.array([3.2, 1.9])
MLGMRES(A, b4, x0, e, nmax_iter)

retraining


[array([ 0.03549643, -0.27095661]),
 array([2.11044698, 1.10697372]),
 array([2.1, 1.1])]

In [27]:
b5 = np.array([2.2, 2.0])
MLGMRES(A, b5, x0, e, nmax_iter)

[array([-0.10228174,  0.91851342]),
 array([1.49325211, 0.62417187]),
 array([1.54285714, 0.65714286])]

In [28]:
b6 = np.array([0.2, 2.0])
MLGMRES(A, b6, x0, e, nmax_iter)

retraining


[array([-0.59030252,  2.18956411]),
 array([ 0.54942084, -0.09750034]),
 array([ 0.4, -0.2])]

In [29]:
b7 = np.array([3.2, 4.0])
MLGMRES(A, b7, x0, e, nmax_iter)

[array([0.40446289, 0.15053272]),
 array([2.33516369, 0.718055  ]),
 array([2.4, 0.8])]

In [30]:
b8 = np.array([3.3, 2.0])
MLGMRES(A, b8, x0, e, nmax_iter)

retraining


[array([-0.01001329, -0.03236413]),
 array([2.18416861, 1.13597938]),
 array([2.17142857, 1.12857143])]

In [31]:
b9 = np.array([5.2, 2.0])
MLGMRES(A, b9, x0, e, nmax_iter)

[array([ 0.16482563, -0.11899924]),
 array([3.16110972, 1.86882464]),
 array([3.25714286, 1.94285714])]

In [32]:
b10 = np.array([4.2, 1.0])
MLGMRES(A, b10, x0, e, nmax_iter)

retraining


[array([ 0.26160986, -0.20421576]),
 array([2.39075618, 1.54066108]),
 array([2.54285714, 1.65714286])]

In [33]:
b10 = np.array([4.2, 1.0])
MLGMRES(A, b10, x0, e, nmax_iter)

[array([ 0.06679101, -0.03977418]),
 array([2.50395164, 1.62665997]),
 array([2.54285714, 1.65714286])]

In [34]:
b10 = np.array([4.2, 1.0])
MLGMRES(A, b10, x0, e, nmax_iter)

retraining


[array([ 0.06679101, -0.03977418]),
 array([2.50395164, 1.62665997]),
 array([2.54285714, 1.65714286])]

In [35]:
MLGMRES.predictor.counter

12

In [41]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 100, 2, 100, 2

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)   # equivalent to b  -- in the context of Ax = b
y = torch.empty(N, D_out)  # equivalent to x  ----------- ~~ ------------
for i in range(N):
    y[i, :] = torch.from_numpy(matmul_a(A, x[i, :]))

# Construct our model by instantiating the class defined above
model = TwoLayerNet(D_in, H, D_out)

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the two
# nn.Linear modules which are members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

n_steps = 10000
print_f = n_steps/100
with tqdm(total=n_steps, file=sys.stdout, leave=True) as pbar:
    for t in range(n_steps):
        # Forward pass: Compute predicted y by passing x to the model
        y_pred = model(x)

        # Compute and print loss
        loss = criterion(y_pred, y)
        if t % print_f == 0:
            # print(t, loss.item())
            loss_val = loss.item()
            pbar.set_description(f"t={t:6d}, loss={loss_val:.5f}")

        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        pbar.update(1)

t=  9900, loss=0.01801: 100%|██████████| 10000/10000 [00:03<00:00, 2706.47it/s]


In [37]:
np.dot(A,x[0,:])

matrix([[-1.23950583, -0.3039158 ]])

In [38]:
model.forward(x[0,:])

tensor([-1.2395, -0.2751], grad_fn=<AddBackward0>)

In [39]:
y[0,:]

tensor([-1.2395, -0.3039])

In [40]:
x[0,:]

tensor([-0.7517, -0.4878])